In [2]:
import os
import cv2
import tempfile
from deepface import DeepFace
from urllib.request import urlopen
import shutil
import pandas as pd
import numpy as np
from PIL import Image, ImageEnhance

## Gets all the unique faces but might have duplicates

In [17]:
def process_video_frames(frame_directory,j, output_directory="unique_faces", 
                         model_name="Facenet", 
                         distance_metric="cosine", 
                         max_unique_faces = 4,
                         threshold=0.6):
    """
    Process frames from a directory to extract and store unique faces using DeepFace
    
    Args:
        frame_directory (str): Path to directory containing video frames
        output_directory (str): Path to store unique faces
        model_name (str): Face recognition model to use
        distance_metric (str): Metric for face comparison
        threshold (float): Similarity threshold for unique face detection
    """
    # Create directories
    os.makedirs(output_directory, exist_ok=True)
    os.makedirs("temp_unique_faces", exist_ok=True)

    # Track unique faces
    unique_faces_count = 0


    # Iterate through frames
    for frame_filename in sorted(os.listdir(frame_directory)):
        if frame_filename.endswith(('.jpg', '.png', '.jpeg')):
            frame_path = os.path.join(frame_directory, frame_filename)
            
            try:
                # Detect faces using DeepFace
                detections = DeepFace.extract_faces(
                    frame_path, 
                    detector_backend="mtcnn", 
                    enforce_detection=True,
                    align=True
                )
                
                print(f"found faces in this frame")
                
                # Process each detected face
                for i, detection in enumerate(detections):
                    if unique_faces_count >= max_unique_faces:
                        return
                    
                    # Extract the face
                    if detection['confidence'] > 0.95:
                        face = detection['face']
                    else:
                        print("low face confidence")
                        continue
                    
                    # Ensure face is valid
                    if face is None or face.size == 0:
                        print(f"Invalid face detected in {frame_filename}")
                        continue
                    
                    # Convert to uint8 if needed
                    if face.dtype != np.uint8:
                        face = (255 * face).astype(np.uint8)
                    
                    # Convert to PIL Image
                    pil_face = Image.fromarray(face)
                    
                    # Enhance face image
                    enhanced_face = enhance_face_image(pil_face)
                    
                    # Temporary path for current face
                    temp_face_path = os.path.join("temp_unique_faces", f"temp_face_{frame_filename}.jpg")
                    enhanced_face.save(temp_face_path)

                    # Check if the image if valid or not i.e it contains a face
                    # try:
                    #     res = DeepFace.extract_faces(img_path=temp_face_path)
                    # except Exception as e:
                    #     continue
                    

                    # Check if the face is unique
                    is_unique = check_unique_face(
                        temp_face_path, 
                        output_directory, 
                        model_name, 
                        distance_metric, 
                        threshold
                    )
                    
                    # Save unique face
                    if is_unique:
                        print("unique face found")
                        unique_filename = f"unique_face_{unique_faces_count}_{j}_{frame_filename}"
                        unique_path = os.path.join(output_directory, unique_filename)
                        print(f"unique_path : {unique_path}")
                        enhanced_face.save(unique_path)
                        unique_faces_count += 1
                        print(f"Saved unique face: {unique_filename}")
                    
                    # Clean up temp face
                    os.remove(temp_face_path)
            
            except Exception as e:
                print(f"Error processing {frame_filename} : No face in this image or frame")

def check_unique_face(face_path, output_directory, model_name, distance_metric, threshold):
    """
    Check if the face is unique compared to existing faces in the output directory
    
    Args:
        face_path (str): Path to the current face image
        output_directory (str): Directory containing existing unique faces
        model_name (str): Face recognition model
        distance_metric (str): Metric for face comparison
        threshold (float): Similarity threshold
    
    Returns:
        bool: True if face is unique, False otherwise
    """
    # If output directory is empty, face is unique
    if not os.listdir(output_directory):
        return True
    
    try:
        # Use DeepFace to find similar faces
        results = DeepFace.find(
            img_path=face_path,
            db_path=output_directory,
            model_name=model_name,
            threshold = 0.8,
            distance_metric=distance_metric
        )
        
        # Check if any similar faces are found
        if results is None or (isinstance(results, pd.DataFrame) and results.empty):
            return True
        
        # If results exist, check the distance
        if isinstance(results, list) and len(results) > 0:
            # Get the minimum distance
            min_distance = results[0]['distance'].min() if not results[0].empty else float('inf')
            return min_distance > threshold
        

        return False
    
    except ValueError as e:
        # If an error occurs (e.g., no faces found), consider it unique
        print(f"Face comparison error: {e}")
        return False

def check_unique_face2(face_path, output_directory, model_name, distance_metric, threshold=0.8):
    """
    Improved unique face checking using multiple verification methods
    """
    # If output directory is empty, face is unique
    existing_faces = os.listdir(output_directory)
    if not existing_faces:
        return True
    
    try:
        # Iterate through existing faces and verify each
        for existing_face in existing_faces:
            existing_face_path = os.path.join(output_directory, existing_face)
            
            try:
                # Use verify method for more precise comparison
                verification_result = DeepFace.verify(
                    img1_path=face_path,
                    img2_path=existing_face_path,                    
                    threshold=0.8  # Adjusted threshold
                )
    
                # If verification returns True (similar face found), it's not unique
                if verification_result['verified']:
                    return False
            
            except Exception as verify_error:
                # If verification fails for a specific image, continue checking others
                print(f"Verification error with {existing_face}: {verify_error}")
                continue
                #return False
        
        # If no similar faces found after checking all existing faces
        return True
    
    except Exception as e:
        print(f"Unique face check error: {e}")
        return False

def enhance_face_image(pil_image, resize_dim=(256, 256)):
    """
    Enhance face image quality
    
    Args:
        pil_image (PIL.Image): Input face image
        resize_dim (tuple): Target resize dimensions
    
    Returns:
        PIL.Image: Enhanced face image
    """
    # Resize image
    resized_image = pil_image.resize(resize_dim, Image.LANCZOS)
    
    # Enhance sharpness
    sharpness_enhancer = ImageEnhance.Sharpness(resized_image)
    sharpened_image = sharpness_enhancer.enhance(2.0)
    
    # Enhance contrast
    contrast_enhancer = ImageEnhance.Contrast(sharpened_image)
    enhanced_image = contrast_enhancer.enhance(1.2)
    
    return enhanced_image

# Function to create the main directory for storing unique faces
def create_main_directory(directory="unique_faces"):
    if not os.path.exists(directory):
        os.makedirs(directory)
    return directory

# Function to create/clear the extracted frames directory
def create_extracted_frames_directory(directory="extracted_frames"):
    if os.path.exists(directory):
        shutil.rmtree(directory)  # Remove existing frames
    os.makedirs(directory)
    return directory

# Function to extract frames from a video
def extract_frames(video_path, frame_count=20):
    frames = []
    cap = cv2.VideoCapture(video_path)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    interval = max(total_frames // frame_count, 1)  # Calculate frame interval
    
    for i in range(frame_count):
        frame_index = i * interval
        cap.set(cv2.CAP_PROP_POS_FRAMES, frame_index)
        ret, frame = cap.read()
        if ret:
            frames.append(frame)
    cap.release()
    return frames
# Function to process videos and extract unique faces
def process_videos(video_urls, main_dir="unique_faces", frame_count=30, model_name="Facenet", distance_metric="cosine", threshold=0.4):
    main_dir = create_main_directory(main_dir)
    
    # Iterate over each video URL
    j = 1
    for video_url in video_urls:
        print(f"Processing video: {video_url}")

        # Create/clear the extracted frames directory for each video
        extracted_frames_dir = create_extracted_frames_directory("extracted_frames")
        
        # Download the video to a temporary file
        with tempfile.NamedTemporaryFile(delete=True, suffix=".mp4") as temp_video:
            temp_video.write(urlopen(video_url).read())
            temp_video.flush()

            # Extract frames from the video
            frames = extract_frames(temp_video.name, frame_count=frame_count)
            
            # Store extracted frames temporarily in the extracted_frames directory
            for frame_index, frame in enumerate(frames):
                frame_path = os.path.join(extracted_frames_dir, f"frame_{frame_index}.jpg")
                cv2.imwrite(frame_path, frame)
            
            # Iterate through extracted frames to detect faces and store unique faces
            process_video_frames(extracted_frames_dir,j)
            
            # Clean up the extracted frames directory after processing
            shutil.rmtree(extracted_frames_dir)
            j += 1

    print("Processing completed.")

# List of video URLs
video_urls = [
    "https://fgimagestorage.blob.core.windows.net/facebook-assets/hd-987643572956494",
    "https://fgimagestorage.blob.core.windows.net/facebook-assets/hd-904174908300812",
    "https://fgimagestorage.blob.core.windows.net/facebook-assets/hd-992064161877405",
    "https://fgimagestorage.blob.core.windows.net/facebook-assets/hd-905739711170399",
    "https://fgimagestorage.blob.core.windows.net/facebook-assets/hd-901695498551491",
    "https://fgimagestorage.blob.core.windows.net/facebook-assets/hd-879631723735807"
]

# Run the process
process_videos(video_urls)


Processing video: https://fgimagestorage.blob.core.windows.net/facebook-assets/hd-987643572956494
1/1 [==============================] - 0s 22ms/step
found faces in this frame
low face confidence
1/1 [==============================] - 0s 14ms/step
found faces in this frame
unique face found
unique_path : unique_faces/unique_face_0_1_frame_1.jpg
Saved unique face: unique_face_0_1_frame_1.jpg
1/1 [==============================] - 0s 13ms/step
found faces in this frame
24-12-01 13:11:32 - Found 1 newly added image(s), 0 removed image(s), 0 replaced image(s).


Finding representations: 100%|██████████| 1/1 [00:00<00:00, 26.83it/s]

24-12-01 13:11:32 - 🔴 Exception while extracting faces from unique_faces/unique_face_0_1_frame_1.jpg: Face could not be detected in unique_faces/unique_face_0_1_frame_1.jpg.Please confirm that the picture is a face photo or consider to set enforce_detection param to False.
24-12-01 13:11:32 - There are now 1 representations in ds_model_facenet_detector_opencv_aligned_normalization_base_expand_0.pkl
24-12-01 13:11:32 - Searching temp_unique_faces/temp_face_frame_10.jpg.jpg in 1 length datastore


24-12-01 13:11:33 - find function duration 0.5161137580871582 seconds
unique face found
unique_path : unique_faces/unique_face_1_1_frame_10.jpg
Saved unique face: unique_face_1_1_frame_10.jpg
low face confidence
1/1 [==============================] - 0s 13ms/step
found faces in this frame
24-12-01 13:11:34 - Found 1 newly added image(s), 0 removed image(s), 0 replaced image(s).


Finding representations: 100%|██████████| 1/1 [00:00<00:00,  8.23it/s]

...

24-12-01 13:13:45 - There are now 8 representations in ds_model_facenet_detector_opencv_aligned_normalization_base_expand_0.pkl
24-12-01 13:13:45 - Searching temp_unique_faces/temp_face_frame_17.jpg.jpg in 8 length datastore
Face comparison error: Face could not be detected in temp_unique_faces/temp_face_frame_17.jpg.jpg.Please confirm that the picture is a face photo or consider to set enforce_detection param to False.
low face confidence


1/1 [==============================] - 0s 15ms/step
found faces in this frame
24-12-01 13:13:46 - Searching temp_unique_faces/temp_face_frame_18.jpg.jpg in 8 length datastore
24-12-01 13:13:46 - find function duration 0.13089799880981445 seconds
unique face found
unique_path : unique_faces/unique_face_1_4_frame_18.jpg
Saved unique face: unique_face_1_4_frame_18.jpg
24-12-01 13:13:46 - Found 1 newly added image(s), 0 removed image(s), 0 replaced image(s).


Finding representations: 100%|██████████| 1/1 [00:00<00:00,  6.44it/s]

24-12-01 13:13:46 - There are now 9 representations in ds_model_facenet_detector_opencv_aligned_normalization_base_expand_0.pkl
24-12-01 13:13:46 - Searching temp_unique_faces/temp_face_frame_18.jpg.jpg in 9 length datastore
Face comparison error: Face could not be detected in temp_unique_faces/temp_face_frame_18.jpg.jpg.Please confirm that the picture is a face photo or consider to set enforce_detection param to False.


1/1 [==============================] - 0s 16ms/step
found faces in this frame
24-12-01 13:13:47 - Searching temp_unique_faces/temp_face_frame_19.jpg.jpg in 9 length datastore
24-12-01 13:13:47 - find function duration 0.11559605598449707 seconds
24-12-01 13:13:47 - Searching temp_unique_faces/temp_face_frame_19.jpg.jpg in 9 length datastore
Face comparison error: Face could not be detected in temp_unique_faces/temp_face_frame_19.jpg.jpg.Please confirm that the picture is a face photo or consider to set enforce_detection param to False.
low face confidence
1/1 [==============================] - 0s 15ms/step
found faces in this frame
24-12-01 13:13:48 - Searching temp_unique_faces/temp_face_frame_2.jpg.jpg in 9 length datastore
Face comparison error: Face could not be detected in temp_unique_faces/temp_face_frame_2.jpg.jpg.Please confirm that the picture is a face photo or consider to set enforce_detection param to False.
low face confidence
1/1 [==============================] - 0s 17ms

## Extremely precise with few to none duplicates but might miss some faces and not accurate

### Basically less duplicates but misses unique faces

In [3]:
def process_video_frames(frame_directory,j, output_directory="unique_faces", 
                         model_name="Facenet", 
                         distance_metric="cosine", 
                         max_unique_faces = 4,
                         threshold=0.8):
    """
    Process frames from a directory to extract and store unique faces using DeepFace
    
    Args:
        frame_directory (str): Path to directory containing video frames
        output_directory (str): Path to store unique faces
        model_name (str): Face recognition model to use
        distance_metric (str): Metric for face comparison
        threshold (float): Similarity threshold for unique face detection
    """
    # Create directories
    os.makedirs(output_directory, exist_ok=True)
    os.makedirs("temp_unique_faces", exist_ok=True)

    # Track unique faces
    unique_faces_count = 0


    # Iterate through frames
    for frame_filename in sorted(os.listdir(frame_directory)):
        if frame_filename.endswith(('.jpg', '.png', '.jpeg')):
            frame_path = os.path.join(frame_directory, frame_filename)
            
            try:
                # Detect faces using DeepFace
                detections = DeepFace.extract_faces(
                    frame_path, 
                    detector_backend="mtcnn", 
                    enforce_detection=True,
                    align=True
                )
                
                print(f"found faces in this frame")
                
                # Process each detected face
                for i, detection in enumerate(detections):
                    if unique_faces_count >= max_unique_faces:
                        return
                    
                    # Extract the face
                    if detection['confidence'] > 0.95:
                        face = detection['face']
                    else:
                        print("low face confidence")
                        continue
                    
                    # Ensure face is valid
                    if face is None or face.size == 0:
                        print(f"Invalid face detected in {frame_filename}")
                        continue
                    
                    # Convert to uint8 if needed
                    if face.dtype != np.uint8:
                        face = (255 * face).astype(np.uint8)
                    
                    # Convert to PIL Image
                    pil_face = Image.fromarray(face)
                    
                    # Enhance face image
                    enhanced_face = enhance_face_image(pil_face)
                    
                    # Temporary path for current face
                    temp_face_path = os.path.join("temp_unique_faces", f"temp_face_{frame_filename}.jpg")
                    enhanced_face.save(temp_face_path)

                    # Check if the image if valid or not i.e it contains a face
                    # try:
                    #     res = DeepFace.extract_faces(img_path=temp_face_path)
                    # except Exception as e:
                    #     continue
                    

                    # Check if the face is unique
                    is_unique = check_unique_face(
                        temp_face_path, 
                        output_directory, 
                        model_name, 
                        distance_metric, 
                        threshold
                    )
                    
                    # Save unique face
                    if is_unique:
                        print("unique face found")
                        unique_filename = f"unique_face_{unique_faces_count}_{j}_{frame_filename}"
                        unique_path = os.path.join(output_directory, unique_filename)
                        print(f"unique_path : {unique_path}")
                        enhanced_face.save(unique_path)
                        unique_faces_count += 1
                        print(f"Saved unique face: {unique_filename}")
                    
                    # Clean up temp face
                    os.remove(temp_face_path)
            
            except Exception as e:
                print(f"Error processing {frame_filename} : No face in this image or frame")

def check_unique_face(face_path, output_directory, model_name, distance_metric, threshold):
    """
    Check if the face is unique compared to existing faces in the output directory
    
    Args:
        face_path (str): Path to the current face image
        output_directory (str): Directory containing existing unique faces
        model_name (str): Face recognition model
        distance_metric (str): Metric for face comparison
        threshold (float): Similarity threshold
    
    Returns:
        bool: True if face is unique, False otherwise
    """
    # If output directory is empty, face is unique
    if not os.listdir(output_directory):
        return True
    
    try:
        # Use DeepFace to find similar faces
        results = DeepFace.find(
            img_path=face_path,
            db_path=output_directory,
            model_name=model_name,
            threshold = 0.8,
            distance_metric=distance_metric
        )
        
        # Check if any similar faces are found
        if results is None or (isinstance(results, pd.DataFrame) and results.empty):
            return True
        
        # If results exist, check the distance
        if isinstance(results, list) and len(results) > 0:
            # Get the minimum distance
            min_distance = results[0]['distance'].min() if not results[0].empty else float('inf')
            return min_distance > threshold
        

        return False
    
    except ValueError as e:
        # If an error occurs (e.g., no faces found), consider it unique
        print(f"Face comparison error: {e}")
        return False

def check_unique_face2(face_path, output_directory, model_name, distance_metric, threshold=0.6):
    """
    Improved unique face checking using multiple verification methods
    """
    # If output directory is empty, face is unique
    existing_faces = os.listdir(output_directory)
    if not existing_faces:
        return True
    
    try:
        # Iterate through existing faces and verify each
        for existing_face in existing_faces:
            existing_face_path = os.path.join(output_directory, existing_face)
            
            try:
                # Use verify method for more precise comparison
                verification_result = DeepFace.verify(
                    img1_path=face_path,
                    img2_path=existing_face_path,                    
                    threshold=0.8  # Adjusted threshold
                )
                
                # If verification returns True (similar face found), it's not unique
                if verification_result['verified']:
                    return False
            
            except Exception as verify_error:
                # If verification fails for a specific image, continue checking others
                print(f"Verification error with {existing_face}: {verify_error}")
                continue
                #return False
        
        # If no similar faces found after checking all existing faces
        return True
    
    except Exception as e:
        print(f"Unique face check error: {e}")
        return False

def enhance_face_image(pil_image, resize_dim=(256, 256)):
    """
    Enhance face image quality
    
    Args:
        pil_image (PIL.Image): Input face image
        resize_dim (tuple): Target resize dimensions
    
    Returns:
        PIL.Image: Enhanced face image
    """
    # Resize image
    resized_image = pil_image.resize(resize_dim, Image.LANCZOS)
    
    # Enhance sharpness
    sharpness_enhancer = ImageEnhance.Sharpness(resized_image)
    sharpened_image = sharpness_enhancer.enhance(2.0)
    
    # Enhance contrast
    contrast_enhancer = ImageEnhance.Contrast(sharpened_image)
    enhanced_image = contrast_enhancer.enhance(1.2)
    
    return enhanced_image

# Function to create the main directory for storing unique faces
def create_main_directory(directory="unique_faces"):
    if not os.path.exists(directory):
        os.makedirs(directory)
    return directory

# Function to create/clear the extracted frames directory
def create_extracted_frames_directory(directory="extracted_frames"):
    if os.path.exists(directory):
        shutil.rmtree(directory)  # Remove existing frames
    os.makedirs(directory)
    return directory

# Function to extract frames from a video
def extract_frames(video_path, frame_count=20):
    frames = []
    cap = cv2.VideoCapture(video_path)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    interval = max(total_frames // frame_count, 1)  # Calculate frame interval
    
    for i in range(frame_count):
        frame_index = i * interval
        cap.set(cv2.CAP_PROP_POS_FRAMES, frame_index)
        ret, frame = cap.read()
        if ret:
            frames.append(frame)
    cap.release()
    return frames
# Function to process videos and extract unique faces
def process_videos(video_urls, main_dir="unique_faces", frame_count=30, model_name="Facenet", distance_metric="cosine", threshold=0.4):
    main_dir = create_main_directory(main_dir)
    
    # Iterate over each video URL
    j = 1
    for video_url in video_urls:
        print(f"Processing video: {video_url}")

        # Create/clear the extracted frames directory for each video
        extracted_frames_dir = create_extracted_frames_directory("extracted_frames")
        
        # Download the video to a temporary file
        with tempfile.NamedTemporaryFile(delete=True, suffix=".mp4") as temp_video:
            temp_video.write(urlopen(video_url).read())
            temp_video.flush()

            # Extract frames from the video
            frames = extract_frames(temp_video.name, frame_count=frame_count)
            
            # Store extracted frames temporarily in the extracted_frames directory
            for frame_index, frame in enumerate(frames):
                frame_path = os.path.join(extracted_frames_dir, f"frame_{frame_index}.jpg")
                cv2.imwrite(frame_path, frame)
            
            # Iterate through extracted frames to detect faces and store unique faces
            process_video_frames(extracted_frames_dir,j)
            
            # Clean up the extracted frames directory after processing
            shutil.rmtree(extracted_frames_dir)
            j += 1

    print("Processing completed.")

# List of video URLs
video_urls = [
    "https://fgimagestorage.blob.core.windows.net/facebook-assets/hd-987643572956494",
    "https://fgimagestorage.blob.core.windows.net/facebook-assets/hd-904174908300812",
    "https://fgimagestorage.blob.core.windows.net/facebook-assets/hd-992064161877405",
    "https://fgimagestorage.blob.core.windows.net/facebook-assets/hd-905739711170399",
    "https://fgimagestorage.blob.core.windows.net/facebook-assets/hd-901695498551491",
    "https://fgimagestorage.blob.core.windows.net/facebook-assets/hd-879631723735807"
]

# Run the process
process_videos(video_urls)


Processing video: https://fgimagestorage.blob.core.windows.net/facebook-assets/hd-987643572956494
1/1 [==============================] - 0s 60ms/step
found faces in this frame
low face confidence
1/1 [==============================] - 0s 14ms/step
found faces in this frame
unique face found
unique_path : unique_faces/unique_face_0_1_frame_1.jpg
Saved unique face: unique_face_0_1_frame_1.jpg
1/1 [==============================] - 0s 12ms/step
found faces in this frame
24-12-01 16:53:44 - Found 1 newly added image(s), 0 removed image(s), 0 replaced image(s).


Finding representations: 100%|██████████| 1/1 [00:00<00:00,  3.86it/s]

24-12-01 16:53:44 - 🔴 Exception while extracting faces from unique_faces/unique_face_0_1_frame_1.jpg: Face could not be detected in unique_faces/unique_face_0_1_frame_1.jpg.Please confirm that the picture is a face photo or consider to set enforce_detection param to False.
24-12-01 16:53:44 - There are now 1 representations in ds_model_facenet_detector_opencv_aligned_normalization_base_expand_0.pkl
24-12-01 16:53:44 - Searching temp_unique_faces/temp_face_frame_10.jpg.jpg in 1 length datastore


24-12-01 16:53:46 - find function duration 1.7460589408874512 seconds
unique face found
unique_path : unique_faces/unique_face_1_1_frame_10.jpg
Saved unique face: unique_face_1_1_frame_10.jpg
low face confidence
1/1 [==============================] - 0s 13ms/step
found faces in this frame
24-12-01 16:53:47 - Found 1 newly added image(s), 0 removed image(s), 0 replaced image(s).


Finding representations: 100%|██████████| 1/1 [00:00<00:00,  8.38it/s]

...

24-12-01 16:55:53 - There are now 4 representations in ds_model_facenet_detector_opencv_aligned_normalization_base_expand_0.pkl
24-12-01 16:55:53 - Searching temp_unique_faces/temp_face_frame_8.jpg.jpg in 4 length datastore


24-12-01 16:55:53 - find function duration 0.28073811531066895 seconds
24-12-01 16:55:53 - Searching temp_unique_faces/temp_face_frame_8.jpg.jpg in 4 length datastore
24-12-01 16:55:53 - find function duration 0.12446904182434082 seconds
1/1 [==============================] - 0s 16ms/step
found faces in this frame
24-12-01 16:55:54 - Searching temp_unique_faces/temp_face_frame_9.jpg.jpg in 4 length datastore
24-12-01 16:55:54 - find function duration 0.13133001327514648 seconds
24-12-01 16:55:54 - Searching temp_unique_faces/temp_face_frame_9.jpg.jpg in 4 length datastore
24-12-01 16:55:55 - find function duration 0.12220096588134766 seconds
Processing video: https://fgimagestorage.blob.core.windows.net/facebook-assets/hd-905739711170399
1/1 [==============================] - 0s 15ms/step
found faces in this frame
24-12-01 16:55:58 - Searching temp_unique_faces/temp_face_frame_0.jpg.jpg in 4 length datastore
Face comparison error: Face could not be detected in temp_unique_faces/temp_fa

Finding representations: 100%|██████████| 1/1 [00:00<00:00,  8.15it/s]

24-12-01 16:56:10 - There are now 5 representations in ds_model_facenet_detector_opencv_aligned_normalization_base_expand_0.pkl
24-12-01 16:56:10 - Searching temp_unique_faces/temp_face_frame_17.jpg.jpg in 5 length datastore
Face comparison error: Face could not be detected in temp_unique_faces/temp_face_frame_17.jpg.jpg.Please confirm that the picture is a face photo or consider to set enforce_detection param to False.
low face confidence


1/1 [==============================] - 0s 17ms/step
found faces in this frame
24-12-01 16:56:11 - Searching temp_unique_faces/temp_face_frame_18.jpg.jpg in 5 length datastore
24-12-01 16:56:11 - find function duration 0.1333460807800293 seconds
24-12-01 16:56:11 - Searching temp_unique_faces/temp_face_frame_18.jpg.jpg in 5 length datastore
Face comparison error: Face could not be detected in temp_unique_faces/temp_face_frame_18.jpg.jpg.Please confirm that the picture is a face photo or consider to set enforce_detection param to False.
1/1 [==============================] - 0s 18ms/step
found faces in this frame
24-12-01 16:56:12 - Searching temp_unique_faces/temp_face_frame_19.jpg.jpg in 5 length datastore
24-12-01 16:56:12 - find function duration 0.16875600814819336 seconds
24-12-01 16:56:12 - Searching temp_unique_faces/temp_face_frame_19.jpg.jpg in 5 length datastore
Face comparison error: Face could not be detected in temp_unique_faces/temp_face_frame_19.jpg.jpg.Please confirm tha

In [16]:
dfs = DeepFace.find(
    img_path = "unique_faces/unique_face_2_1_frame_26.jpg",
    db_path = "unique_faces"
   
)

24-12-01 00:32:56 - Searching unique_faces/unique_face_2_1_frame_26.jpg in 9 length datastore
24-12-01 00:32:56 - find function duration 0.35715603828430176 seconds
